In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from chess import Board
import os
import tensorflow as tf
import time

# inspired by Github user Skripkon 
# https://github.com/Skripkon/chess-engine/blob/main/engines/tensorflow/train_and_predict.ipynb
# this script can play n number of games
# for any players you want that we have availabe to us 

In [ ]:
# Load in the model that you want to use
# and the move encoding

model = tf.keras.models.load_model("models/simulated_filtered_model/SSMF_50EPOCHS.keras")
import pickle

with open("models/simulated_filtered_model/move_to_int.pkl", "rb") as f:
    move_to_int = pickle.load(f)

with open("models/simulated_filtered_model/int_to_move.pkl", "rb") as f:
    int_to_move = pickle.load(f)


In [ ]:
#translating the board into a matrix for the predition process 
def board_to_matrix(board: Board):
    matrix = np.zeros((8, 8, 12))
    piece_map = board.piece_map()
    for square, piece in piece_map.items():
        row, col = divmod(square, 8)
        piece_type = piece.piece_type - 1
        piece_color = 0 if piece.color else 6
        matrix[row, col, piece_type + piece_color] = 1
    return matrix

In [ ]:
import chess
import chess.engine
import chess.pgn
import random

# stockfish engine path
# this is the path on my Home machine it should be chnaged if you are using it 
STOCKFISH_PATH = "/opt/homebrew/bin/stockfish"
stockfish_engine = chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH)

# engine stats
stockfish_engine.configure({
    "UCI_LimitStrength": True,
    "UCI_Elo": 2000
})

# stockfish model move
def stockfish_move(board):
    result = stockfish_engine.play(board, chess.engine.Limit(time=0.1))
    return result.move.uci()

# our model move
def predict_next_move(board):
    board_matrix = board_to_matrix(board).reshape(1, 8, 8, 12)
    predictions = model.predict(board_matrix, verbose = 0)[0]
    legal_moves = list(board.legal_moves)
    legal_moves_uci = [move.uci() for move in legal_moves]
    sorted_indices = np.argsort(predictions)[::-1]
    for move_index in sorted_indices:
        move = int_to_move[move_index]
        if move in legal_moves_uci:
            return move
    return None


# Counters
wins = 0
losses = 0
draws = 0 

# Run n games
for i in range(100):
    # make a game 
    board = chess.Board()
    game = chess.pgn.Game()
    node = game

    #play a game
    while not board.is_game_over():
        if board.turn == chess.WHITE:
            move_uci = predict_next_move(board)
        else:
            move_uci = stockfish_move(board)

        move = chess.Move.from_uci(move_uci)
        board.push(move)
        node = node.add_variation(move)

    outcome = board.outcome()

    if outcome is not None:
        if outcome.winner == chess.WHITE:
            wins += 1
            
        elif outcome.winner == chess.BLACK:
            losses += 1
        else: 
            draws += 1
    # if you want to the see the game string        
    #print(game)

# Final results
print("Wins:", wins)
print("Losses:", losses)

# Cleanup
stockfish_engine.quit()
